In [1]:
print("This is the test for initial commit!")

This is the test for initial commit!


In [2]:
import pandas as pd
print("Pandas imported successfully, version: "+pd.__version__)
import statsmodels as sm
print("Statsmodels imported successfully, version: "+sm.__version__)

#Plotly packages
import plotly.graph_objects as go
print("Plotly function imported succesfully")
import plotly.express as px
print("Plotly express imported succesfully")
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.figure_factory as ff

#Matplotlib packages
import matplotlib.pyplot as plt

#Numpy + Statistics
import numpy as np
print("Numpy imported successfully, version: "+np.__version__)

from statsmodels.graphics.gofplots import qqplot

#Define needed function in the code
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

Pandas imported successfully, version: 0.25.1
Statsmodels imported successfully, version: 0.10.1
Plotly function imported succesfully
Plotly express imported succesfully
Numpy imported successfully, version: 1.17.2


In [ ]:
#Import dataset

data_all = pd.read_csv('/Users/mazutislab/Desktop/SynBio/US_Accidents_May19.csv')

In [ ]:
#Changing max display columns, in order to see all columns
pd.set_option('display.max_columns', 50)
data_all.head()

In [ ]:
#Selecting only needed data in the given dataset
data_cleaned = data_all[["Severity", "Temperature(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)", "Wind_Speed(mph)", 
                        "Precipitation(in)"]]
data_cleaned.reset_index()
type(data_cleaned)

#Convert Temperature in Fahrenheit to Celsius
def fahr_to_celsius(temp_fahr):
    """Convert Fahrenheit to Celsius and Return Celsius conversion of input"""
    temp_celsius = (temp_fahr - 32) * 5 / 9
    return temp_celsius

data_cleaned["Temperature(C)"] = (fahr_to_celsius(data_cleaned["Temperature(F)"])).round(2)
data_cleaned.drop(['Temperature(F)'], inplace = True, axis = 1)

#Convert Pressure in inches of mercury to mbar
data_cleaned["Pressure(mbar)"] = data_cleaned["Pressure(in)"]*0.033863886666667*1000
data_cleaned.drop(['Pressure(in)'], inplace = True, axis = 1)

#Convert Wind Speed in mph to kmh
data_cleaned["Wind_Speed(kmh)"] = data_cleaned["Wind_Speed(mph)"]*1.609344
data_cleaned.drop(['Wind_Speed(mph)'], inplace = True, axis = 1)

#Convert Precipitation in inches to mm
data_cleaned["Precipitation(mm)"] = data_cleaned["Precipitation(in)"]*25.4
data_cleaned.drop(['Precipitation(in)'], inplace = True, axis = 1)

#Convert Visibility in miles to km
data_cleaned["Visibility(km)"] = data_cleaned["Visibility(mi)"]*1.609344
data_cleaned.drop(['Visibility(mi)'], inplace = True, axis = 1)
data_cleaned.head()

#Slice data, so that Severity >= 1
data_cleaned = data_cleaned[data_cleaned['Severity'] >= 1]
data_cleaned = data_cleaned.sort_values(by=['Severity'], ascending = True)
data_cleaned.reset_index()
data_cleaned.head()

In [ ]:
#Descriptive Statistics

data_cleaned.describe()

In [ ]:
#Histogram for Severity groups

fig = go.Figure(data=[go.Histogram(x=(data_cleaned['Severity']))])

fig.show()

#Frequency of severities
Severity_frequency = {}

for i, u in enumerate(range(0,5)):
    Severity_frequency['Severity' + str(u)] = (data_cleaned[(data_cleaned['Severity']==i)]).shape[0]

print(Severity_frequency)


In [ ]:
#Dataset slicing function - Universal - for outliers removal

column_name = input("Enter column name without brackets. Possible choices: " + str(list(data_cleaned.columns)) + " : ")

lower_value = int(input("Enter lowest included value in your chosen column for your new dataset "))
higher_value = int(input("Enter highest included value in your chosen column for your new dataset "))
data_cleaned_bool = data_cleaned[column_name].between(lower_value, higher_value, inclusive = True)
data_cleaned_new = data_cleaned[data_cleaned_bool]
print(str(((data_cleaned[data_cleaned[column_name] > higher_value]).shape[0])+
          ((data_cleaned[data_cleaned[column_name] < lower_value]).shape[0])) + 
      " measurements have been removed from the dataset \n")


print(data_cleaned_new.head())

print("\n Your new compiled dataset is available under the name of data_cleaned_new")

In [ ]:
#Code to check the number of values in defined interval

#Select dataset (data_cleaned or data_cleaned_new [from slicing code])
data_input = data_cleaned_new

column_name = input("Enter column name without brackets. Possible choices: " + str(list(data_cleaned.columns)) 
                    + " : ")
#Select values
value_1 = int(input("Select lower interval value: "))
value_2 = int(input("Select higher interval value: "))

values_in_interval = data_input[column_name].between(value_1, value_2, inclusive = True)
print("\033[1m There are " + str((data_input[values_in_interval]).shape[0]) + " " + str(column_name) +
      " values between " + str(value_1) + " and " + str(value_2) + " in the given data")

In [ ]:
#Histogram Universal Code

#Select dataset you want to work with (data_cleaned or data_cleaned_new [from slicing code])
data_input = data_cleaned_new

column_name = input("Enter column name without brackets. Possible choices: " + str(list(data_cleaned.columns)) + " : ")
hist_bins = int(input("Enter number of bins in histogram "))

hist_variable = go.Figure(data=[go.Histogram(x=(data_input[column_name]), nbinsx=hist_bins)])

hist_variable.update_layout(height = 450, width = 800,
    xaxis_title=column_name,
    yaxis_title="Frequency"
)

hist_variable.show()


#Lowest/Highest Value
print("Lowest " + column_name + " in dataset is " + str((data_input[column_name]).min()))

print("Highest " + column_name + " in dataset is " + str((data_input[column_name]).max()))



#Histograms Between Severity Groups

sev1 = ((data_input[(data_input['Severity']) == 1])[column_name])
sev2 = ((data_input[(data_input['Severity']) == 2])[column_name])
sev3 = ((data_input[(data_input['Severity']) == 3])[column_name])
sev4 = ((data_input[(data_input['Severity']) == 4])[column_name])

fig = make_subplots(rows=2, cols=2, subplot_titles=("Severity 1","Severity 2", "Severity 3", "Severity 4"))

#Adding Traces
fig.add_trace(go.Histogram(x=(sev1), nbinsx=hist_bins), row=1, col=1)
fig.add_trace(go.Histogram(x=(sev2), nbinsx=hist_bins), row=1, col=2)
fig.add_trace(go.Histogram(x=(sev3), nbinsx=hist_bins), row=2, col=1)
fig.add_trace(go.Histogram(x=(sev4), nbinsx=hist_bins), row=2, col=2)

# Update xaxis properties
fig.update_xaxes(title_text=column_name, row=1, col=1)
fig.update_xaxes(title_text=column_name, row=1, col=2)
fig.update_xaxes(title_text=column_name, row=2, col=1)
fig.update_xaxes(title_text=column_name, row=2, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=2, col=2)

# Update title and height
fig.update_layout(title_text="Variable " + column_name + " histograms for each Severity group", height=800)

fig.show()

In [ ]:
#Box Plot Universal

#Choose data input (data_cleaned or data_cleaned_new [from slicing code])
data_input = data_cleaned_new

box_plot_temperature = px.box(data_input, x="Severity", y=column_name)
box_plot_temperature.show()


#Median + Mean Calculation for displaying with box plots
Median_results = {}
Mean_results = {}
sd_results = {}
sem_results= {}

for i, u in enumerate(range(1,5), 1):
    Median_results['Severity' + str(u)] = (((data_input[data_input['Severity'] == i])[column_name]).median())
    Mean_results['Severity' + str(u)] = (((data_input[data_input['Severity'] == i])[column_name]).mean())
    sd_results['Severity' + str(u)] = (((data_input[data_input['Severity'] == i])[column_name]).std())
    sem_results['Severity' + str(u)] = (((data_input[data_input['Severity'] == i])[column_name]).sem())

    
Mean_pd = pd.DataFrame.from_dict(Mean_results, orient='index')
Mean_pd.columns = ['Mean']

Median_pd = pd.DataFrame.from_dict(Median_results, orient='index')
Median_pd.columns = ['Median']

sd_pd = pd.DataFrame.from_dict(sd_results, orient='index')
sd_pd.columns = ['Standard Deviation']

sem_pd = pd.DataFrame.from_dict(sem_results, orient = 'index')
sem_pd.columns = ['Standard Error']

result = Mean_pd.join(Median_pd)

result2 = result.join(sd_pd)

result_final = result2.join(sem_pd)

print(result_final)




In [ ]:
#Anderson-Darling Test Universal

from scipy.stats import normaltest
from scipy.stats import anderson

#Choose data input (data_cleaned or data_cleaned_new [from slicing code])
data_input = data_cleaned_new

column_name = input("Enter column name without brackets. Possible choices: " + 
                    str(list(data_cleaned.columns)) + " : ")

#Get numpy arrays of variable values for each Severity groups

sev1_and = ((data_input[(data_input['Severity']) == 1])[column_name])
Severity1 = sev1_and.to_numpy()
Severity1 = Severity1[np.logical_not(np.isnan(Severity1))]

sev2_and = ((data_input[(data_input['Severity']) == 2])[column_name])
Severity2 = sev2_and.to_numpy()
Severity2 = Severity2[np.logical_not(np.isnan(Severity2))]

sev3_and = ((data_input[(data_input['Severity']) == 3])[column_name])
Severity3 = sev3_and.to_numpy()
Severity3 = Severity3[np.logical_not(np.isnan(Severity3))]

sev4_and = ((data_input[(data_input['Severity']) == 4])[column_name])
Severity4 = sev4_and.to_numpy()
Severity4 = Severity4[np.logical_not(np.isnan(Severity4))]

sev_list = [Severity1, Severity2, Severity3, Severity4]

#Element-wise iteration through all Severity groups

counter = 0

for i in sev_list:
    result = anderson(i)
    print("\033[1m" + (namestr(i, globals())[0] + ' Statistic: %.3f \033[0m' % result.statistic))
    while counter <1:
        p = 0
        for i in range(len(result.critical_values)):
            sl, cv = result.significance_level[i], result.critical_values[i]
            if result.statistic < result.critical_values[i]:
                print('Significance level: %.3f: Critical Value : %.3f, data looks normal (fail to reject H0)' % (sl, cv))
            else:
                print('Significance level: %.3f: Critical Value : %.3f, data does not look normal (reject H0)' % (sl, cv))
            counter += 1
    counter = 0

In [ ]:
#ANOVA universal

import scipy.stats as stats

#Choose data input (data_cleaned or data_cleaned_new [from slicing code])
data_input = data_cleaned_new

column_name = input("Enter column name without brackets. Possible choices: " + 
                    str(list(data_cleaned.columns)) + " : ")


#Get numpy arrays of variable values for each Severity groups

sev1_anova_no_outliers = ((data_cleaned_new[(data_cleaned_new['Severity']) == 1])[column_name])
Severity1_no_outliers = sev1_anova_no_outliers.to_numpy()
Severity1_no_outliers = Severity1_no_outliers[np.logical_not(np.isnan(Severity1_no_outliers))]

sev2_anova_no_outliers = ((data_cleaned_new[(data_cleaned_new['Severity']) == 2])[column_name])
Severity2_no_outliers = sev2_anova_no_outliers.to_numpy()
Severity2_no_outliers = Severity2_no_outliers[np.logical_not(np.isnan(Severity2_no_outliers))]

sev3_anova_no_outliers = ((data_cleaned_new[(data_cleaned_new['Severity']) == 3])[column_name])
Severity3_no_outliers = sev3_anova_no_outliers.to_numpy()
Severity3_no_outliers = Severity3_no_outliers[np.logical_not(np.isnan(Severity3_no_outliers))]

sev4_anova_no_outliers = ((data_cleaned_new[(data_cleaned_new['Severity']) == 4])[column_name])
Severity4_no_outliers = sev4_anova_no_outliers.to_numpy()
Severity4_no_outliers = Severity4_no_outliers[np.logical_not(np.isnan(Severity4_no_outliers))]

#ANOVA test

F, p = stats.f_oneway(Severity1_no_outliers, Severity2_no_outliers, Severity3_no_outliers, Severity4_no_outliers)
print(str(column_name) + ' variable ANOVA value F is equal to ' + str(F))
print(str(column_name) + ' variable ANOVA value p is equal to ' + str(p))

In [ ]:
#Kruskal Wallis Universal


#Choose data input (data_cleaned or data_cleaned_new [from slicing code])
data_input = data_cleaned_new

column_name = input("Enter column name without brackets. Possible choices: " + 
                    str(list(data_cleaned.columns)) + " : ")


#Get numpy arrays of variable values for each Severity groups

sev1_anova_no_outliers = ((data_cleaned_new[(data_cleaned_new['Severity']) == 1])[column_name])
Severity1_no_outliers = sev1_anova_no_outliers.to_numpy()
Severity1_no_outliers = Severity1_no_outliers[np.logical_not(np.isnan(Severity1_no_outliers))]

sev2_anova_no_outliers = ((data_cleaned_new[(data_cleaned_new['Severity']) == 2])[column_name])
Severity2_no_outliers = sev2_anova_no_outliers.to_numpy()
Severity2_no_outliers = Severity2_no_outliers[np.logical_not(np.isnan(Severity2_no_outliers))]

sev3_anova_no_outliers = ((data_cleaned_new[(data_cleaned_new['Severity']) == 3])[column_name])
Severity3_no_outliers = sev3_anova_no_outliers.to_numpy()
Severity3_no_outliers = Severity3_no_outliers[np.logical_not(np.isnan(Severity3_no_outliers))]

sev4_anova_no_outliers = ((data_cleaned_new[(data_cleaned_new['Severity']) == 4])[column_name])
Severity4_no_outliers = sev4_anova_no_outliers.to_numpy()
Severity4_no_outliers = Severity4_no_outliers[np.logical_not(np.isnan(Severity4_no_outliers))]

#Kruskal-Wallis test

from scipy.stats import kruskal

stat, p = kruskal(Severity1_no_outliers, Severity2_no_outliers, Severity3_no_outliers, Severity4_no_outliers)
print(str(column_name) + ' variable Kruskal-Wallis Statistics value is equal to ' + str(stat))
print(str(column_name) + ' variable Kruskal-Wallis p value is equal to ' + str(p))

In [ ]:
#Z-test Universal

from statsmodels.stats import weightstats as stests

#Choose data input (data_cleaned or data_cleaned_new [from slicing code])
data_input = data_cleaned_new

column_name = input("Enter column name without brackets. Possible choices: " + 
                    str(list(data_cleaned.columns)) + " : ")
    
#Get numpy arrays of variable values for each Severity groups

sev1_and = ((data_input[(data_input['Severity']) == 1])[column_name])
Severity1 = sev1_and.to_numpy()
Severity1 = Severity1[np.logical_not(np.isnan(Severity1))]

sev2_and = ((data_input[(data_input['Severity']) == 2])[column_name])
Severity2 = sev2_and.to_numpy()
Severity2 = Severity2[np.logical_not(np.isnan(Severity2))]

sev3_and = ((data_input[(data_input['Severity']) == 3])[column_name])
Severity3 = sev3_and.to_numpy()
Severity3 = Severity3[np.logical_not(np.isnan(Severity3))]

sev4_and = ((data_input[(data_input['Severity']) == 4])[column_name])
Severity4 = sev4_and.to_numpy()
Severity4 = Severity4[np.logical_not(np.isnan(Severity4))]


sev_all = [Severity1, Severity2, Severity3, Severity4]
sev_all2 = [Severity2, Severity3, Severity4]

#Create new dictionaries for z test

z_statistics={}
z_p_values={}


#Element-wise iteration with all Severity groups
counter = 0

for i in sev_all:
    while sev_all2 and counter <1:
        for u in sev_all2:
            ztest, pval = stests.ztest(i, u, value=0, alternative='two-sided')
            if pval == 1 or ztest == 0:
                pass
            else:
                z_p_values[namestr(i, globals())[1]+ " vs " + namestr(u, globals())[1]]=pval
                z_statistics[namestr(i, globals())[1]+ " vs " + namestr(u, globals())[1]]=ztest
                counter += 1
    counter = 0
    

z_test_pd1 = pd.DataFrame.from_dict(z_statistics, orient='index')
z_test_pd1.columns = ['Z Statistics']

z_test_pd2 = pd.DataFrame.from_dict(z_p_values, orient='index')
z_test_pd2.columns = ['Z P Value']

result_z = z_test_pd1.join(z_test_pd2)

result_z = result_z.drop([result_z.index[5], result_z.index[7], result_z.index[8]])
print(result_z)

In [ ]:
#Mann Whitney U non parametric

from scipy.stats import mannwhitneyu


#Choose data input (data_cleaned or data_cleaned_new [from slicing code])
data_input = data_cleaned_new

column_name = input("Enter column name without brackets. Possible choices: " + 
                    str(list(data_cleaned.columns)) + " : ")

sev1_and = ((data_input[(data_input['Severity']) == 1])[column_name])
Severity1 = sev1_and.to_numpy()
Severity1 = Severity1[np.logical_not(np.isnan(Severity1))]

sev2_and = ((data_input[(data_input['Severity']) == 2])[column_name])
Severity2 = sev2_and.to_numpy()
Severity2 = Severity2[np.logical_not(np.isnan(Severity2))]

sev3_and = ((data_input[(data_input['Severity']) == 3])[column_name])
Severity3 = sev3_and.to_numpy()
Severity3 = Severity3[np.logical_not(np.isnan(Severity3))]

sev4_and = ((data_input[(data_input['Severity']) == 4])[column_name])
Severity4 = sev4_and.to_numpy()
Severity4 = Severity4[np.logical_not(np.isnan(Severity4))]


sev_all = [Severity1, Severity2, Severity3, Severity4]
sev_all2 = [Severity2, Severity3, Severity4]


#U TEST

#Create empty dictonaries
U_stat_values={}
U_p_values={}
    
    
#Iterate element-wise
counter = 0

for i in sev_all:
    while counter <1:
        for u in sev_all2:
            stat, p = mannwhitneyu(i, u)
            if pval == 1 or ztest == 0:
                pass
            else:
                U_p_values[namestr(i, globals())[1]+ " vs " + namestr(u, globals())[1]]=p
                U_stat_values[namestr(i, globals())[1]+ " vs " + namestr(u, globals())[1]]=stat
                counter += 1
    counter = 0
    

U_stat_pd1 = pd.DataFrame.from_dict(U_stat_values, orient='index')
U_stat_pd1.columns = ['Statistics']

U_p_pd2 = pd.DataFrame.from_dict(U_p_values, orient='index')
U_p_pd2.columns = ['P Value']

result_U = U_stat_pd1.join(U_p_pd2)

result_U = result_U.drop([result_U.index[3], result_U.index[6], result_U.index[8], result_U.index[9]])
print(result_U)

In [ ]:
##Function (I want to leave it here, in case I need it later)
#
#list = ["a", "b", "c", "d"]
#list2 = ["b", "c", "d"]
#
#counter = 0
#
#for i in list:
#    while counter <1:
#        for u in list2:
#            if i == u:
#                pass
#            else:
#                print(i+u) 
#            counter += 1
#    counter = 0

In [ ]:
#Cleaning data_set for ordinal regression (choosing only specific values)

data_cleaned_final = data_cleaned[data_cleaned['Temperature(C)'].between(-55, 50, inclusive = True)]

data_cleaned_final = data_cleaned_final[data_cleaned_final['Pressure(mbar)'].between(950, 1055, inclusive = True)]

data_cleaned_final = data_cleaned_final[data_cleaned_final['Wind_Speed(kmh)'].between(0, 150, inclusive = True)]

data_cleaned_final = data_cleaned_final[data_cleaned_final['Visibility(km)'].between(0, 80, inclusive = True)]

print(data_cleaned_final)

In [ ]:
column_name = input("Enter column name without brackets. Possible choices: " + 
                    str(list(data_cleaned.columns)) + " : ")

data_sev_temp = data_cleaned_final[['Severity', column_name]]
                          
data_sev_temp = data_sev_temp.dropna()

data_sev_temp = data_sev_temp.sort_values(by=['Severity'])

#Extracting only numpy array with variable values

data_sev_temp2 = data_sev_temp.drop(columns='Severity')


#Getting ordinal variable numpy array for each variable value

data_sev_temp = (data_sev_temp['Severity']).to_numpy()

#Ordinal Logistic Regression

from bevel.linear_ordinal_regression import OrderedLogit

ls = OrderedLogit()
ls.fit(data_sev_temp2, data_sev_temp)

ls.print_summary()

#Probability for each value to be in specific Severity group

result = ls.predict_probabilities(data_sev_temp2)

print(result[:][3])

In [ ]:
#Probability to be in Severity 4 analysis - UNIVERSAL

column_name = input("Enter column name without brackets. Possible choices: " + 
                    str(list(data_cleaned.columns)) + " : ")

#Extract array of column_name variable values

variable_array = np.array(data_sev_temp2[column_name])
print(variable_array)

#Extract Probability Array Values

list_res = []

for i in range(0, len(result)):
   list_res.append(result[i][3])

Probability_array = np.array(list_res)
print(Probability_array)


#Linear Regression

import numpy as np
from sklearn.linear_model import LinearRegression

variable_array_regression = variable_array.reshape((-1, 1))

model = LinearRegression().fit(variable_array_regression, Probability_array)

print(model)

r_sq = model.score(variable_array_regression, Probability_array)
intercept = model.intercept_
slope = model.coef_


#Line for plotting

line = slope*variable_array+intercept
print(line)

#Create DataFrame from column_name variable and Probability

pd_result = pd.DataFrame({column_name:variable_array, 'Probability':Probability_array})
print(pd_result)


In [ ]:
#Plotting probability in Severity group 4 and line chart of fitted linear regression model

fig = px.scatter(pd_result, x=column_name, y="Probability")
fig.add_trace(
    go.Scatter(
        x=variable_array,
        y=line,
        mode="lines",
        line=go.scatter.Line(color="gray"),
        showlegend=False)
)
fig.show()

print('Coefficient of determination (R^2 value): ' + str(r_sq))
print('Slope : ' + str(slope))
print('Intercept : ' + str(intercept))